# Практика 8: Методы защиты от атак на модели ИИ

##  Цель задания:
 Изучить методы защиты моделей ИИ от различных атак, включая методы защиты на уровне данных, моделирования и обучения. Реализовать эти методы и проверить их эффективность против атак, изученных ранее.

###  Задачи:
 1. Изучить и реализовать защиту модели с помощью тренировок на противоречивых примерах
 (Adversarial Training).
 2. Реализовать метод защиты на основе градиентной маскировки.
 3. Использовать регуляризацию и нормализацию для повышения устойчивости модели.
 4. Проверить эффективность методов защиты против атак FGSM, PGD и GAN-based атак.
 5. Оценить улучшение точности защищенной модели на противоречивых примерах.

Выполнил студент ББМО-02-23 Евдокимов А.М.

##  Шаги выполнения:

###  Шаг 1: Защита с помощью Adversarial Training
 Adversarial Training — это метод защиты, который заключается в том, чтобы обучать модель на противоречивых примерах. Этот метод помогает модели научиться быть более устойчивой к атакам, так как она сталкивается с противоречивыми примерами на этапе обучения.

In [19]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0
# Преобразование меток в one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)
# Уменьшаем количество обучающих изображений до 1000
train_images = train_images[:1000]
train_labels = train_labels[:1000]

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * np.sign(gradient)
    adversarial_image = image + perturbation
    adversarial_image = np.clip(adversarial_image, 0, 1)
    return adversarial_image
# Функция для генерации противоречивых примеров
def generate_adversarial_examples(model, images, labels, epsilon):
  adversarial_images = []
  for i in range(len(images)):
    image = tf.convert_to_tensor(images[i].reshape((1, 28, 28)))
    label = labels[i]
    if len(label.shape) > 1 and label.shape[1] > 1:
        label = np.argmax(label),
    label = tf.convert_to_tensor(label)
    with tf.GradientTape() as tape:
      tape.watch(image)
      prediction = model(image)
      loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)
    gradient = tape.gradient(loss, image)
    adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
    adversarial_images.append(adversarial_image.reshape(28, 28))
  return np.array(adversarial_images)
# Создание модели
def create_model():
  model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model
# Обучение модели с противоречивыми примерами
def adversarial_training(model, train_images, train_labels, epsilon):
 for epoch in range(5): # Короткое обучение для демонстрации
  print(f'Epoch:{epoch}')
  for i in range(0, len(train_images), 64):
    batch_images = train_images[i:i+64]
    batch_labels = train_labels[i:i+64]
    # Генерация противоречивых примеров для текущей партии данных
    adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
    # Объединение оригинальных и противоречивых примеров
    combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
    combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
    # Обучение на комбинированных данных
    model.train_on_batch(combined_images, combined_labels)
# Инициализация модели
model = create_model()
# Тренировка модели с защитой на противоречивых примерах
adversarial_training(model, train_images, train_labels, epsilon=0.1) # ограничил тренировочные данные для ускорения обучения
# Сохранение защищенной модели
model.save('adversarial_trained_model.h5')

Epoch:0
Epoch:1
Epoch:2
Epoch:3
Epoch:4


In [22]:
# Оценка модели
loss, accuracy = model.evaluate(train_images, train_labels, verbose=1)
print(f'Final Loss: {loss:.4f}')
print(f'Final Accuracy: {accuracy:.4f}')


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9675 - loss: 0.1575
Final Loss: 0.1575
Final Accuracy: 0.9690


### Шаг 2: Градиентная маскировка (Gradient Masking)
 Gradient Masking — это метод защиты, который затрудняет доступ к градиентам модели для атак. Он
 используется для уменьшения информации, доступной для атакующих, и усложнения поиска
 направленных изменений.

In [7]:
# Реализация градиентной маскировки
 # Для демонстрации мы можем использовать специальные функции активации
from tensorflow.keras.layers import Activation
# Обновление модели для градиентной маскировки
def create_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),
        Activation('softplus')  # Используем softplus вместо softmax для градиентной маскировки
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
# Обучение модели с градиентной маскировкой
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с градиентной маскировкой
masked_model.save('masked_model.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3913 - loss: 1.9495
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8394 - loss: 0.8530
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8965 - loss: 0.4451
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9279 - loss: 0.3113
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9522 - loss: 0.2508


### Шаг 3: Регуляризация и нормализация для повышения устойчивости
 Использование таких методов, как L2-регуляризация, дропаут и нормализация батчей, может помочь улучшить устойчивость модели к атакам.

In [23]:
# Модель с регуляризацией и нормализацией
def create_regularized_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu',
kernel_regularizer=tf.keras.regularizers.l2(0.01)),
 tf.keras.layers.Dropout(0.5),tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(10, activation='softmax')
 ])
 model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
 return model
# Обучение модели с регуляризацией и нормализацией
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с регуляризацией
regularized_model.save('regularized_model.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2376 - loss: 4.3077
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6936 - loss: 2.3098
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8073 - loss: 1.7402
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8561 - loss: 1.4697
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8792 - loss: 1.3173


### Шаг 4: Оценка моделей на противоречивых примерах

Теперь проверим эффективность всех защитных методов на атакованных данных, созданных с помощью FGSM и других методов, таких как PGD или GAN.

In [24]:
# Загрузка атакованной модели
protected_model = tf.keras.models.load_model('adversarial_trained_model.h5')
# Генерация противоречивых примеров для тестовых данных
adversarial_test_images = generate_adversarial_examples(protected_model, test_images, test_labels, epsilon=0.1)
# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = protected_model.evaluate(adversarial_test_images, test_labels)
print(f'Accuracy of protected model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4482 - loss: 1.5693
Accuracy of protected model on adversarial examples: 0.5116000175476074


### Шаг 5: Сравнение методов защиты

In [32]:
# Оценка модели с Adversarial Training
print("Adversarially Trained Model Accuracy on Adversarial Examples:")
adv_loss, adv_acc = protected_model.evaluate(adversarial_test_images, test_labels, verbose=1)

# Оценка модели с Gradient Masking
print("Masked Model Accuracy on Adversarial Examples:")
masked_loss, masked_acc = masked_model.evaluate(adversarial_test_images, test_labels, verbose=1)

# Оценка модели с Регуляризацией и нормализацией
print("Regularized Model Accuracy on Adversarial Examples:")
reg_loss, reg_acc = regularized_model.evaluate(adversarial_test_images, test_labels, verbose=1)


Adversarially Trained Model Accuracy on Adversarial Examples:
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4482 - loss: 1.5693
Masked Model Accuracy on Adversarial Examples:
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4344 - loss: 1.7021
Regularized Model Accuracy on Adversarial Examples:
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5559 - loss: 2.2379


In [34]:
# Сравнение результатов
results = {
    "Adversarial Training": {"Loss": adv_loss, "Accuracy": adv_acc},
    "Gradient Masking": {"Loss": masked_loss, "Accuracy": masked_acc},
    "Regularization and Normalization": {"Loss": reg_loss, "Accuracy": reg_acc},
}
print("\nComparison of Defense Methods:")
for method, metrics in results.items():
    print(f"{method}: Loss = {metrics['Loss']:.4f}, Accuracy = {metrics['Accuracy']:.4f}")



Comparison of Defense Methods:
Adversarial Training: Loss = 1.4053, Accuracy = 0.5116
Gradient Masking: Loss = 1.5258, Accuracy = 0.4836
Regularization and Normalization: Loss = 2.1311, Accuracy = 0.6021


Комбинирование методов
Комбинирование Adversarial Training и Регуляризации может быть наиболее эффективным:

Adversarial Training повышает устойчивость к FGSM.
Регуляризация и нормализация улучшают общую генерализацию модели и снижают переобучение.

In [35]:
def create_combined_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu',
                              kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение комбинированной модели
combined_model = create_combined_model()
adversarial_training(combined_model, train_images, train_labels, epsilon=0.1)


Epoch:0
Epoch:1
Epoch:2
Epoch:3
Epoch:4


In [37]:
# Оценка модели
loss, accuracy = combined_model.evaluate(train_images, train_labels, verbose=1)
print(f'Final Loss: {loss:.4f}')
print(f'Final Accuracy: {accuracy:.4f}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9016 - loss: 1.5004
Final Loss: 1.5083
Final Accuracy: 0.9010


Влияние увеличенного epsilon на FGSM

In [42]:
# Тестирование при различных уровнях epsilon
epsilons = [0.01, 0.025, 0.05, 0.075, 0.1]
for eps in epsilons:
    adversarial_test_images = generate_adversarial_examples(protected_model, test_images, test_labels, epsilon=eps)
    loss, acc = protected_model.evaluate(adversarial_test_images, test_labels, verbose=0)
    print(f"Epsilon: {eps:.4f} | Loss: {loss:.4f} | Accuracy: {acc:.4f}")


Epsilon: 0.0100 | Loss: 0.6268 | Accuracy: 0.8159
Epsilon: 0.0250 | Loss: 0.7402 | Accuracy: 0.7695
Epsilon: 0.0500 | Loss: 0.9451 | Accuracy: 0.6883
Epsilon: 0.0750 | Loss: 1.1673 | Accuracy: 0.6035
Epsilon: 0.1000 | Loss: 1.4053 | Accuracy: 0.5116


### Выводы

Adversarial Training (Точность: 44.82%, Потери: 1.5693): Модель, обученная с использованием противоречивых примеров, демонстрирует умеренную устойчивость к FGSM-атакам. Хотя её точность выше, чем у модели с градиентной маскировкой, потери остаются высокими. Это говорит о том, что Adversarial Training эффективен, но может не справляться с более сложными или неизвестными типами атак.

Gradient Masking (Точность: 43.44%, Потери: 1.7021):
Модель с градиентной маскировкой показывает чуть более низкую точность и немного большие потери, чем Adversarial Training. Это подтверждает, что Gradient Masking затрудняет генерацию атак, но не устраняет их воздействие полностью. Метод может быть уязвим к адаптивным атакам, которые обойдут градиентные ограничения.

Регуляризация и нормализация (Точность: 55.59%, Потери: 2.2379):
Модель с регуляризацией и нормализацией демонстрирует наивысшую точность среди всех методов защиты, но и самые большие потери. Это свидетельствует о её способности лучше классифицировать атакованные примеры, но ценой высоких потерь. Такой метод может быть полезен в условиях, где критична точность, но не строгое подавление атак.


Лучший результат по точности: Метод регуляризации и нормализации, который достигает 55.59%. Однако, высокие потери показывают, что модель не полностью устойчива к FGSM-атакам.
Умеренная устойчивость: Adversarial Training, обеспечивающий сбалансированную защиту с точностью 44.82% и умеренными потерями.
Ограниченная эффективность: Gradient Masking, демонстрирующий чуть меньшую точность (43.44%) и незначительно большие потери (1.7021), чем Adversarial Training.

Эффективность методов:

Объединение Adversarial Training и Регуляризации даёт наилучшие результаты для FGSM и других атак.

Уровень шума (epsilon):

С увеличением epsilon точность всех моделей падает, но модели с Adversarial Training показывают меньший спад точности.
Этот подход можно использовать для глубокого анализа и выработки рекомендаций по повышению устойчивости модели. Если нужна дополнительная помощь, напишите!